In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# pytorch
import torch

#detectron2
from detectron2.data.datasets import register_coco_instances
import detectron2.data.transforms as T
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_train_loader, DatasetMapper
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.evaluation.evaluator import DatasetEvaluator


#python
import os
import random
import numpy as np
from pathlib import Path

#computer vision
import cv2


# pycocotools
import pycocotools.mask as mask_util

## Steps
1. register coco instance 
2. get configfile and set configfile
3. prepare trainer
4. train


## Others
1. add in augmentation


## Config File

In [ ]:
class CONFIG:
    seed = 42
    
    # cocojson 
    train_coco_json = "../input/satorius-segmentation-coco-json/train_coco.json"
    val_coco_json = "../input/satorius-segmentation-coco-json/val_coco.json"
    image_root_dir = "../input/sartorius-cell-instance-segmentation/"
    train_dataset_name = "satorius_train"
    val_dataset_name = "satorius_val"
    num_dataloader_workers = 2
    
    # detetron2 model zoo model
    model_zoo_model ="COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
    num_classes = 3
    roi_head_batch_size_per_image =128
    
    #solver
    learning_rate = 0.0005 
    solver_steps = (2500, 4500)
    max_iter = 8000
    

## Seed everything

In [ ]:
def seed_torch(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
def seed_python(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_python(CONFIG.seed)
seed_torch(CONFIG.seed)  

## Registering dataset

In [ ]:
# register train_data
register_coco_instances(CONFIG.train_dataset_name, {}, CONFIG.train_coco_json, CONFIG.image_root_dir)
# register val data
register_coco_instances(CONFIG.val_dataset_name, {}, CONFIG.val_coco_json, CONFIG.image_root_dir)

#metadatas
train_metadata = MetadataCatalog.get(CONFIG.train_dataset_name)
val_metadata = MetadataCatalog.get(CONFIG.val_dataset_name)

## Setup config file

In [ ]:
# get default config
cfg = get_cfg() 

# model config
cfg.merge_from_file(model_zoo.get_config_file(CONFIG.model_zoo_model)) # This have to be done first 
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG.model_zoo_model)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = CONFIG.num_classes
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = CONFIG.roi_head_batch_size_per_image
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5


# dataset config
cfg.INPUT.MASK_FORMAT = 'bitmask'
cfg.DATASETS.TRAIN = (CONFIG.train_dataset_name,)
cfg.DATASETS.TEST = (CONFIG.val_dataset_name,)
cfg.DATALOADER.NUM_WORKERS = CONFIG.num_dataloader_workers



# Solver
cfg.SOLVER.CHECKPOINT_PERIOD = 800
cfg.SOLVER.IMS_PER_BATCH = 2 # single gpu
cfg.SOLVER.BASE_LR = CONFIG.learning_rate
# cfg.SOLVER.STEPS = CONFIG.solver_steps
cfg.SOLVER.MAX_ITER = CONFIG.max_iter

# Evaluator
cfg.TEST.EVAL_PERIOD = 800


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## Prepare trainer

https://www.kaggle.com/slawekbiel/positive-score-with-detectron-2-3-training \
https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-detectron2-training#Training
https://ortegatron.medium.com/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e

- augmentation
- dataloader
- trainer

## Overwrite build_train_loaders with augmentations
List of available augmentations can be found here
https://detectron2.readthedocs.io/en/latest/modules/data_transforms.html

# Custom evaluator
https://ortegatron.medium.com/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e \ 
https://www.kaggle.com/slawekbiel/positive-score-with-detectron-2-3-training


In [ ]:
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class CustomEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

In [ ]:
class CustomTrainer(DefaultTrainer):
#     @classmethod
#     def build_train_loader(cls,cfg):
#         return build_detection_train_loader(cfg, 
#                                             mapper = DatasetMapper(cfg, is_train=True, augmentations=[
#                                                 T.RandomBrightness(0.8,1.2),
#                                                 T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
#                                                 T.RandomFlip(prob=0.5, horizontal= False, vertical=True),
#                                                 T.RandomSaturation(0.8,1.2),
#                                                 T.RandomLighting(0.8)
#                                             ]))
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return CustomEvaluator(dataset_name)

## Training using default trainer

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()